In [42]:
from langchain.memory import ConversationKGMemory
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(temperature=0.1)

memory = ConversationKGMemory(
    llm=llm,
    return_messages=True,
)


def add_message(input, output):
    memory.save_context({"input": input}, {"output": output})


add_message(
    "Hi I'm Maverick, I'm in Topgun. My rival is Iceman", "Wow, that is so cool!"
)
add_message("Do you have anything to ask?", "yes, how's Iceman doing?")
add_message(
    "I don't know. I thought that we are friend but we fought yesterday again, yesterday.",
    "I thought that we are getting better, what happened?",
)
add_message(
    "Same as usual, I thought that we became a friend but...",
    "Look, Iceman is comming!",
)
add_message(
    "Oh, shit! What should I do.",
    "Say something! You told me he is your friend and also a wingman.",
)
memory.load_memory_variables({"input": "What is Maverick's relationship with Iceman?"})

{'history': [SystemMessage(content='On Maverick: Maverick is in Topgun. Maverick has a rival named Iceman.')]}

In [ ]:
add_message(
    "Oh, shit. What should I do. Oh, hi, Iceman, you look good today.",
    "Say something!",
)

get_history()